In [27]:
import sqlite3
conn = sqlite3.connect('../DataSets/Wikibooks/wikibooks.sqlite')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[]


In [28]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('../DataSets/Wikibooks/wikibooks.sqlite')

# Get the column names of the English subset
cursor = conn.execute("SELECT * FROM en LIMIT 1")
col_names = [description[0] for description in cursor.description]
print(col_names)

OperationalError: no such table: en

In [29]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to the SQLite database
conn = sqlite3.connect('../DataSets/Wikibooks/wikibooks.sqlite')

# WE ARE LIMITING THE DATA TO ONLY 100 ROWS SINCE THE DATASET IS HUGE
# Load the data into a pandas dataframe
df = pd.read_sql_query("SELECT * FROM en LIMIT 100", conn)

DatabaseError: Execution failed on sql 'SELECT * FROM en LIMIT 100': no such table: en

In [30]:
from wordcloud import WordCloud

# Word cloud of the most common words in the 'body_text' column
wordcloud = WordCloud(width=800, height=800, background_color='white', max_words=200).generate(' '.join(df['body_text']))
plt.figure(figsize=(8,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Most common words in the dataset')
plt.show()

NameError: name 'df' is not defined

In [31]:
# Bar chart of the frequency of the top words
plt.figure(figsize=(12,6))
top_words = pd.Series(' '.join(df['body_text']).split()).value_counts()[:20]
sns.barplot(x=top_words.values, y=top_words.index)
plt.title('Frequency of top words in the dataset')
plt.xlabel('Frequency')
plt.ylabel('Words')
plt.show()

# Histogram of the length of the 'body_text' column
plt.figure(figsize=(10,6))
sns.histplot(df['body_text'].apply(len), kde=False)
plt.title('Distribution of the length of articles')
plt.xlabel('Length of articles')
plt.ylabel('Number of articles')
plt.show()


# Scatter plot of the length of 'body_text' by 'title'
plt.figure(figsize=(10,6))
plt.scatter(df['title'].apply(len), df['body_text'].apply(len))
plt.title('Relationship between the length of title and the length of article')
plt.xlabel('Length of title')
plt.ylabel('Length of article')
plt.show()

NameError: name 'df' is not defined

<Figure size 1200x600 with 0 Axes>

In [32]:
# Boxplot of article lengths
plt.figure(figsize=(10,6))
plt.boxplot(df['body_text'].apply(len), vert=False)
plt.title('Distribution of article lengths')
plt.xlabel('Length of articles')
plt.yticks([])
plt.show()

NameError: name 'df' is not defined

<Figure size 1000x600 with 0 Axes>

In [33]:
import networkx as nx
from collections import Counter

# Word co-occurrence network graph
words = ' '.join(df['body_text']).split()
word_counts = Counter(words)
edges = []
for i in range(len(words)-1):
    edges.append((words[i], words[i+1]))
edge_counts = Counter(edges)
top_edges = dict(edge_counts.most_common(100))
G = nx.DiGraph()
for edge in top_edges.keys():
    G.add_edge(edge[0], edge[1], weight=top_edges[edge])
plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, k=0.3)
nx.draw_networkx_nodes(G, pos, node_size=100, node_color='blue', alpha=0.5)
nx.draw_networkx_edges(G, pos, edge_color='red', width=list(top_edges.values()), alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=16, font_family='sans-serif')
plt.axis('off')
plt.title('Word co-occurrence network graph')
plt.show()

NameError: name 'df' is not defined

In [34]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
import nltk
nltk.data.path.append('/kaggle/working/nltk_data')
nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')
nltk.download('punkt', download_dir='/kaggle/working/nltk_data')
nltk.download('stopwords', download_dir='/kaggle/working/nltk_data')

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /kaggle/working/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
import pandas as pd
import sqlite3
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# # Connect to the SQLite database
# conn = sqlite3.connect('/kaggle/input/wikibooks-dataset/wikibooks.sqlite')

# # Load the data into a pandas dataframe
# df = pd.read_sql_query("SELECT * FROM en", conn, chunksize=100)
# df = pd.concat(df, ignore_index=True)

# Preprocess the 'body_text' column
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text.lower())

    # Remove stop words
    words = [word for word in words if not word in stop_words]

    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join words back into a string
    return ' '.join(words)

df['processed_text'] = df['body_text'].apply(preprocess_text)

# Perform NLP tasks on the preprocessed text
# Calculate the frequency distribution of words
words = word_tokenize(' '.join(df['processed_text']))
freq_dist = nltk.FreqDist(words)
print(freq_dist.most_common(20))

NameError: name 'df' is not defined

In [37]:
def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text.lower())

    # Remove stop words
    words = [word for word in words if not word in stop_words]

    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join words back into a string
    return ' '.join(words)

df['processed_text'] = df['body_text'].apply(preprocess_text)

# Named entity recognition
ner_tags = []
for text in df['processed_text']:
    words = word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    ner_tags.append(nltk.ne_chunk(pos_tags))
df['ner_tags'] = ner_tags

NameError: name 'df' is not defined

In [ ]:
fvc